# Lecture 5: Basis Vectors and Coordinate Systems

[![Watch the Video](https://img.shields.io/badge/Watch%20on%20YouTube-FF0000?style=for-the-badge&logo=youtube&logoColor=white)](https://youtube.com/your-channel)

In this lecture, we'll explore how different sets of basis vectors provide alternative ways to describe the same vector space, and why this matters in machine learning.

## Learning Objectives
- Understand what makes a set of vectors a basis
- Learn how to change between different coordinate systems
- Apply basis transformations to real data
- Understand the relationship between basis vectors and feature engineering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
%matplotlib inline

def plot_basis_transformation(original_basis, new_basis, vector, figsize=(15, 5)):
    """
    Plot a vector in both the standard basis and a new basis
    
    Parameters:
    -----------
    original_basis: list of 2D numpy arrays
        The standard basis vectors [e1, e2]
    new_basis: list of 2D numpy arrays
        The new basis vectors [v1, v2]
    vector: 2D numpy array
        The vector to plot in both coordinate systems
    """
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=figsize)
    
    # Plot in standard basis
    ax1.quiver(0, 0, original_basis[0][0], original_basis[0][1], 
               angles='xy', scale_units='xy', scale=1, color='r', label='e₁')
    ax1.quiver(0, 0, original_basis[1][0], original_basis[1][1], 
               angles='xy', scale_units='xy', scale=1, color='b', label='e₂')
    ax1.quiver(0, 0, vector[0], vector[1], angles='xy', scale_units='xy', 
               scale=1, color='g', label='v')
    ax1.grid(True)
    ax1.set_title('Standard Basis')
    ax1.legend()
    ax1.axis('equal')
    
    # Plot in new basis
    ax2.quiver(0, 0, new_basis[0][0], new_basis[0][1], 
               angles='xy', scale_units='xy', scale=1, color='r', label='v₁')
    ax2.quiver(0, 0, new_basis[1][0], new_basis[1][1], 
               angles='xy', scale_units='xy', scale=1, color='b', label='v₂')
    ax2.quiver(0, 0, vector[0], vector[1], angles='xy', scale_units='xy', 
               scale=1, color='g', label='v')
    ax2.grid(True)
    ax2.set_title('New Basis')
    ax2.legend()
    ax2.axis('equal')
    
    # Plot both together
    ax3.quiver(0, 0, original_basis[0][0], original_basis[0][1], 
               angles='xy', scale_units='xy', scale=1, color='r', alpha=0.3)
    ax3.quiver(0, 0, original_basis[1][0], original_basis[1][1], 
               angles='xy', scale_units='xy', scale=1, color='b', alpha=0.3)
    ax3.quiver(0, 0, new_basis[0][0], new_basis[0][1], 
               angles='xy', scale_units='xy', scale=1, color='r')
    ax3.quiver(0, 0, new_basis[1][0], new_basis[1][1], 
               angles='xy', scale_units='xy', scale=1, color='b')
    ax3.quiver(0, 0, vector[0], vector[1], angles='xy', scale_units='xy', 
               scale=1, color='g')
    ax3.grid(True)
    ax3.set_title('Combined View')
    ax3.axis('equal')
    
    plt.tight_layout()
    return fig, (ax1, ax2, ax3)

## 1. What is a Basis?

A basis is a set of linearly independent vectors that span a vector space. For an n-dimensional space, we need exactly n linearly independent vectors.

Key properties:
1. Linear independence (no redundancy)
2. Spanning (completeness)
3. Minimal spanning set

The standard basis in 2D is:
- e₁ = [1, 0]
- e₂ = [0, 1]

In [ ]:
# Define standard basis
e1 = np.array([1, 0])
e2 = np.array([0, 1])
standard_basis = [e1, e2]

# Define a new basis (rotated 45 degrees)
theta = np.pi/4  # 45 degrees
v1 = np.array([np.cos(theta), np.sin(theta)])
v2 = np.array([-np.sin(theta), np.cos(theta)])
new_basis = [v1, v2]

# Vector to represent in both bases
vector = np.array([2, 1])

# Plot the vector in both coordinate systems
plot_basis_transformation(standard_basis, new_basis, vector)
plt.show()

## 2. Change of Basis

To change the representation of a vector from one basis to another, we need:
1. The vector in the original basis
2. The new basis vectors expressed in the original basis
3. A way to solve for the new coordinates

The formula is:
$\mathbf{v} = c_1\mathbf{v}_1 + c_2\mathbf{v}_2$

where $c_1$ and $c_2$ are the coordinates in the new basis.

In [ ]:
def change_of_basis_matrix(new_basis):
    """Create the change of basis matrix from standard basis to new basis"""
    return np.column_stack(new_basis)

def change_coordinates(vector, new_basis):
    """Find coordinates of a vector in a new basis"""
    change_matrix = change_of_basis_matrix(new_basis)
    new_coords = np.linalg.solve(change_matrix, vector)
    return new_coords

# Example: Find coordinates in the rotated basis
new_coords = change_coordinates(vector, new_basis)
print("Vector in standard coordinates:", vector)
print("Vector in new basis coordinates:", new_coords)

# Verify by reconstructing the original vector
reconstructed = new_coords[0] * v1 + new_coords[1] * v2
print("\nReconstructed vector:", reconstructed)
print("Original vector:", vector)
print("Reconstruction error:", np.linalg.norm(vector - reconstructed))

## 3. Applications in Machine Learning

### Feature Transformation
Changing basis is similar to feature transformation in ML:
- PCA finds a new basis that maximizes variance
- Wavelet transforms find bases good for representing signals
- Kernel methods implicitly work in transformed bases

In [ ]:
# Generate some 2D data with correlation
np.random.seed(42)
n_samples = 200
x = np.random.normal(0, 1, n_samples)
y = x + 0.5 * np.random.normal(0, 1, n_samples)
data = np.column_stack([x, y])

# Plot original data
plt.figure(figsize=(15, 5))

plt.subplot(131)
plt.scatter(data[:, 0], data[:, 1], alpha=0.5)
plt.quiver(0, 0, 1, 0, angles='xy', scale_units='xy', scale=1, color='r', label='e₁')
plt.quiver(0, 0, 0, 1, angles='xy', scale_units='xy', scale=1, color='b', label='e₂')
plt.title('Data in Standard Basis')
plt.axis('equal')
plt.grid(True)
plt.legend()

# Compute PCA
from sklearn.decomposition import PCA
pca = PCA()
transformed_data = pca.fit_transform(data)

# Plot transformed data
plt.subplot(132)
plt.scatter(transformed_data[:, 0], transformed_data[:, 1], alpha=0.5)
plt.quiver(0, 0, pca.components_[0, 0], pca.components_[0, 1], 
           angles='xy', scale_units='xy', scale=1, color='r', label='PC1')
plt.quiver(0, 0, pca.components_[1, 0], pca.components_[1, 1], 
           angles='xy', scale_units='xy', scale=1, color='b', label='PC2')
plt.title('Data in PCA Basis')
plt.axis('equal')
plt.grid(True)
plt.legend()

# Combined view
plt.subplot(133)
plt.scatter(data[:, 0], data[:, 1], alpha=0.3, label='Original')
plt.scatter(transformed_data[:, 0], transformed_data[:, 1], alpha=0.3, label='Transformed')
plt.quiver(0, 0, 1, 0, angles='xy', scale_units='xy', scale=1, color='r', alpha=0.3)
plt.quiver(0, 0, 0, 1, angles='xy', scale_units='xy', scale=1, color='b', alpha=0.3)
plt.quiver(0, 0, pca.components_[0, 0], pca.components_[0, 1], 
           angles='xy', scale_units='xy', scale=1, color='r')
plt.quiver(0, 0, pca.components_[1, 0], pca.components_[1, 1], 
           angles='xy', scale_units='xy', scale=1, color='b')
plt.title('Combined View')
plt.axis('equal')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

print("Explained variance ratio:", pca.explained_variance_ratio_)

## 4. Practice Exercises

1. Create a new basis and express some vectors in it
2. Implement a function to check if a set of vectors forms a basis
3. Find a basis for the subspace spanned by a set of vectors
4. Apply PCA to a real dataset and interpret the principal components

Write your solutions in the cell below:

In [ ]:
# Your solution here


## Next Steps

In the next lecture, we'll explore norms and distances, which will help us understand how to measure vectors and the spaces between them.

### Preparation for Next Lecture
1. Review basis and coordinate transformations
2. Think about different ways to measure distance
3. Consider why we might need different ways to measure vectors

### Additional Resources
- [Interactive Basis Transformation Tool](../../resources/visualizations/basis_transform.html)
- [Basis and Coordinates Cheat Sheet](../../resources/cheat_sheets/basis_coordinates.pdf)
- [3Blue1Brown: Change of basis](https://www.3blue1brown.com/lessons/change-of-basis)